In [2]:
# !nvidia smi

In [3]:
!pip3 install transformers

In [4]:
!pip3 install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 50.9 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer,BertModel
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from pylab import rcParams
# from torchmetrics.functional import accuracy
from transformers import get_linear_schedule_with_warmup,get_constant_schedule,get_cosine_schedule_with_warmup
from matplotlib import rc
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  # will print 'cpu' if no GPU is available


cuda


In [19]:
df=pd.read_csv("/content/train.csv", on_bad_lines="skip",engine="python")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,4a16050093a99092,"No, you have to prove that I can't.",0,0,0,0,0,0
1,6bc2122dc27e275a,Re: My major edit summaries \n\nI have receive...,0,0,0,0,0,0
2,5570f7207bade92f,I depersonalized it. It's sad how you don't ob...,0,0,0,0,0,0
3,a17d6408ffb98139,Thats because I was blocked before I could dis...,0,0,0,0,0,0
4,11b1d25d77ffd834,"I apologise for this, I was just angry with ha...",0,0,0,0,0,0


In [20]:
train_df,val_df=train_test_split(df,test_size=0.2,random_state=42)

In [21]:
LABEL_COLUMNS=['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [22]:
train_df[LABEL_COLUMNS].sum()

,0
toxic,7284
severe_toxic,770
obscene,4003
threat,236
insult,3751
identity_hate,678


In [23]:
train_df = train_df.sample(n=100_000, replace=True, random_state=42)


In [24]:
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
92892,e7538e3762f1f5c6,"""\n\nActually, I think you're confusing primar...",0,0,0,0,0,0
93378,46ec6417460059f2,"No, what has no purpose is this conversation. ...",0,0,0,0,0,0
46741,ed4d7b8b3facd04f,)\nIt belongs in the introduction per the manu...,0,0,0,0,0,0
33316,2a36229a6d683ae1,Stop nominating the page for deletion you igno...,1,0,0,0,0,0
74419,4ae86afab0565487,"""the template will be removed, article is not ...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
78781,32dbd3f0d42ff5a8,You clame you want the section to be more nutr...,0,0,0,0,0,0
50169,8ef8f37922b046e6,The only one who has been edit warring is you....,0,0,0,0,0,0
71631,dc848c12d6971ec9,"Guettarda, you made some questionable edits of...",0,0,0,0,0,0
19858,6971765faeefc563,Regarding edits made during August 27 2006 (UT...,0,0,0,0,0,0


In [25]:
sample_row = df.iloc[16]
sample_comment = sample_row['comment_text']
sample_comment_dict = {'comment_text': sample_comment}
print(sample_comment_dict)

{'comment_text': '"\n\n March 2010 \n\n Please do not vandalize pages, as you did with this edit to Rocky Mountains. If you continue to do so, you will be blocked from editing.    "'}


In [28]:
BERT_MODEL_NAME="bert-based-uncased"
tokenizer=BertTokenizer.from_pretrained(BERT_MODEL_NAME)
encoding=tokenizer.encode_plus(sample_comment,add_special_tokens=True,return_tensor="pt")

OSError: bert-based-uncased is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [27]:
encoding.keys()

NameError: name 'encoding' is not defined

In [ ]:
encoding['input_ids'].shape,encoding['attention_mask'].shape

In [ ]:
encoding['input_ids'].squeeze()[:20]

In [ ]:
## converting input_ids to tokens
tokens=tokenizer.convert_ids_to_tokens(encoding['input_ids'].squeeze())
tokens[:20]

In [ ]:
class ToxicCommentDataset(Dataset):
  def __init__(self,data:pd.DataFrame,tokenizer:BertTokenizer,max_len:int=128):
    self.tokenizer=tokenizer
    self.data=data
    self.max_len=max_len
  def __len__(self):
    return len(self.data)
  def __getitem__(self,index:int):
    data_row=self.data.iloc[index]
    comment=data_row.comment_text
    labels=data_row[LABEL_COLUMNS]
    encoding=self.tokenizer.encode_plus(comment,add_special_tokens=True,max_length)
    return dict(comment=comment,input_ids=encoding['input_ids'],attention_mask=encoding['attention_mask'],labels=torch.FloatTensor(labels))


In [ ]:
train_dataset=ToxicCommentDataset(train_df,tokenizer)
sample_item=train_dataset[0]
sample_item.keys()
sample_item['comment_text']
sample_item['labels']
sample_item['input_ids'].shape

In [ ]:
bert_model=BertModel.from_pretrained(BERT_MODEL_NAME,return_dict=True)
sample_item=train_dataset[0]
sample_item.keys()
prediction=bert_model(input_ids=torch.tensor(sample_item['input_ids']).unsqueeze(dim=0),attention_mask=torch.tensor(sample_item['attention_mask']).unsqueeze(dim=0))
prediction.last_hidden_state.shape,prediction.pooler_output.shape

In [ ]:
class ToxicCommentDataModule(pl.LightningDataModule):
  def __init__(self,train_df:pd.DataFrame,test_df:pd.DataFrame,tokenizer,batch_size=8,max_token_len=128):
    super().__init__()
    self.train_df=train_df
    self.test_df=test_df
    self.tokenizer=tokenizer
    self.batch_size=batch_size
    self.max_token_len=max_token_len
  def setup(self,stage=None):
    self.train_dataset=ToxicCommentDataset(self.train_df,self.tokenizer)
    self.test_dataset=ToxicCommentDataset(self.test_df,self.tokenizer)
  def train_dataloader(self):
    return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True,num_workers=4)
  def val_dataloader(self):
    return DataLoader(self.test_dataset,batch_size=self.batch_size,shuffle=True,num_workers=4)
  def test_dataloader(self):
    return DataLoader(self.test_dataset,batch_size=self.batch_size,shuffle=True,num_workers=4)

In [ ]:
data_module=ToxicCommentDataModule(train_df,val_df,tokenizer)
data_module.setup()